In [1]:
# In this homework, we will use well-known Titanic dataset which contains 
# information about passengers of Titanic. The dataset consists of personal 
# information about each passenger and indicator whether the passenger 
# survived. We will use this data to analyse passenger list and their chance for
# survival.

# The provided dataset contains the following attributes:
#  'Age' - age in years,
#  'Fare' - fare ticked price,
#  'Name' - passenger name,
#  'Parch' - # of parents/children of a person on board,
#  'PassengerId' - identifier,
#  'Pclass' - travelling class, 1 = 1. class, 2 = 2. class, 3 = 3. class,
#  'Sex' - sex,
#  'SibSp' - # siblings/spouses on board,
#  'Survived' - 0 = died, 1 = survived,
#  'Embarked' - boarding port C = Cherbourg, Q = Queenstown, S = Southampton,
#  'Cabin' - cabin number
#  'Ticket' - ticket number

In [2]:
import pandas as pd
import numpy

In [3]:
def load_dataset(train_file_path: str, test_file_path: str) -> pd.DataFrame:
    """
    Write a function which loads CSV from two files to pandas DataFrame and
    performs several data processing steps. Use data provided in `data`
    directory for testing ('data/train.csv' as input parameter
    `train_file_path`, and 'data/test.csv'  as `test_file_path`). 
    
    Add column name "Label" to each DataFrame. The column should contain value "Train"
    for data from `train_file_path` and "Test" from test_file_path.
    
    Perform following operations with DataFrames (keep the order of the
    operations):
        1. Concatenate both DataFrames.
        2. Remove columns  "Ticket", "Embarked", "Cabin" from created DataFrame.
        3. Set the index to unique numbers from zero to the number of rows.

    The return value of the function is processed DataFrame.
    """

    # read data from file
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)

    
    # add column "Label"
    train_df["Label"] = 'Train'
    test_df["Label"] = 'Test'


    # 1. Concatenate both DataFrames.
    data = pd.concat([train_df, test_df])


    # 2. Remove columns  "Ticket", "Embarked", "Cabin" from created DataFrame.
    data.drop(["Ticket", "Embarked", "Cabin"], axis=1, inplace=True)


    # 3. Set the index to unique numbers from zero to the number of rows.
    data.reset_index(inplace=True)
    
    data.drop(["index"],axis=1, inplace=True)
    

    return data

In [4]:
def get_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    When working and analysing data, one often needs to deal with missing
    values. For example, some passengers did not fill information about
    family members. In that case, one needs to be aware of it as it may
    introduce bias to the data.

    Write a function which determines the number of missing values in given
    DataFrame. 
    
    The function should output a new DataFrame. The new DataFrame
    should be indexed by columns of original DataFrame. Columns of returned
    DataFrame will be (keep the order of the columns):
        1. "Total" - contains the number of missing values
        2. "Percent" - contains the percentage of missing values with regard to all
        rows of given DataFrame.

    Sort the resulting DataFrame based on the number of missing values from
    largest to smallest.
    
    Example of output:

               |  Total  |  Percent
    "Column1"  |   34.5  |    76.54321
    "Column2"  |   0     |    0
    """

    # create empty dataframe
    df_new = pd.DataFrame()

    # fill new df indexes as columns names of given df
    df_new["index"] = df.columns
    df_new.set_index('index', inplace=True)
    df_new.index.set_names([""], inplace=True)

    # add columns
    df_new["Total"] = 0
    df_new["Percent"] = 0

    # fill columns Total and Percent
    for i in range(0,len(df.columns)):
        df_new.iloc[i, 0] = df.iloc[:, i].isna().sum()

        df_new.iloc[i, 1] = (100 * df.iloc[:, i].isna().sum())/len(df.index)


  
    # sort df
    df_new.sort_values(by=['Total'], ascending=False, inplace=True)
    
    return df_new


In [5]:
def substitute_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    One way how to handle missing data is to substitute missing values with
    some statistic of other rows. We will use this method for two columns:
        1. "Age" - fill missing values with the mean of other rows.
        2. "Fare" - fill missing values with the lowest price of ~$15 (we
        suppose that the majority of unregistered tickets were the cheapest
        ones).

    Do not to modify given DataFrame but create a copy of it.
    """

    # Do not to modify given DataFrame but create a copy of it.
    df_copy = df.copy()
    
    
    # 1. "Age" - fill missing values with the mean of other rows.
    df_copy["Age"].fillna(value=df_copy["Age"].mean(), inplace=True)
    
    
    # 2. "Fare" - fill missing values with the lowest price of ~$15 (we
    # suppose that the majority of unregistered tickets were the cheapest
    # ones).
    df_copy["Fare"].fillna(value=df_copy["Fare"].min(), inplace=True)
    
    return df_copy

In [6]:
def get_correlation(df: pd.DataFrame) -> float:
    """
    We want to know whether there is a relationship between the age of a
    passenger and fare ticket price (e.g. younger children have cheaper
    tickets). We will use Pearson correlation coefficient to quantify linear
    relationship between columns "Age" and "Fare".
    The result will be returned as one number.

    Pearson correlation coefficient quantifies linear relationship between
    two random variables. Correlation ranges from -1 to 1. Value around zero
    indicates no linear relationship, -1 indicates strong negative
    relationship, 1 indicates strong relationship.
    """
    
    return df["Age"].corr(df["Fare"])



In [7]:
def get_survived_per_class(df: pd.DataFrame,
                           group_by_column_name: str) -> pd.Series:
    """
    We want to know how big was the chance of survival for different groups of
    passengers (e.g. for different sexes, classes, etc.). 
    
    Write a function
    that estimates that. The input of the function is a DataFrame with data
    and name of column (group_by_column_name) which holds group information.
    To increase readability of the result sort values from the highest chance of
    survival to lowest and round the resulting values to 2 decimal places.
    Return result as pandas Series.
    
    Example:

    get_survived_per_class(df, "Sex")

                  Survived
    Female     |      0.82
    Male       |      0.32

    """
    
    # Create DataFrame for filling
    series = pd.DataFrame(columns=['index', "Survived"])
    series['index'] = df[group_by_column_name].unique()
    series['Survived'] = series['Survived'].astype("float") 
    
    
    # sort values for right access
    series.sort_values(by=['index'])
    
    
    # filling DataFrame
    row_count = len(df[group_by_column_name])
                    
    index_series = list(df[group_by_column_name].unique())
    index_series.sort() # sort values for right access
    
    index_int = 0
    
    # we could use group by function!!!
    for i in index_series:
        ones = 0
        count = 0
        
        for j in range(0, row_count):
            if i == df[group_by_column_name][j]:
                count += 1
                    
                if df["Survived"][j] == 1:
                    ones +=1
                    
        if(count != 0):
            series.iloc[index_int,1] =  round(ones / count, 2)
        else:
            series.iloc[index_int,1] =  round(0, 2)
        index_int += 1
    
    # convert DataFrame to Series
    series.set_index('index', inplace = True)
    series.index.set_names([""], inplace=True)
    
    series = series.iloc[:,0]
    
    # sort Series
    series = series.sort_values(ascending=False)
    
    
    return series

In [8]:
def get_outliers(df: pd.DataFrame) -> (int, pd.DataFrame):
    """
    We want to explore fare ticket prices. An important part of such
    exploration is exploration of outliers. An outlier may indicate an error
    in the data (somebody entered price incorrectly) or some special group of
    passengers.

    We will use the IQR method for the identification of outliers. IQR method
    considers an outlier any point which does not fulfil:
        Q1 - 1.5*IQR < point_value < Q3 + 1.5*IQR,
    where Q1 and Q3 are the first and the third quartiles respectively
    calculated from all points in data. IQR is the inter-quartile range
    calculate as the difference between Q3 and Q1:
        IQR = Q3 - Q1.

    Return tuple with the number of outliers and all passengers with outlier
    fare ticket price.
    """

    q1=df['Fare'].quantile(0.25)

    q3=df['Fare'].quantile(0.75)

    IQR=q3-q1

    outliers = df[((df['Fare']<(q1-1.5*IQR)) | (df['Fare']>(q3+1.5*IQR)))]
    
    outliers = outliers[outliers['Fare'].notna()]
    
    
    return (len(outliers.iloc[:,0]), outliers)

In [9]:
def create_new_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    To analyse data and use them for modeling, it may be convenient to create
    a new columns (features). These new features are usually created
    transformation of original values. For example, if we want to compare
    survivals from Titanic and SS Eastland we will want to scale fare prices
    to the same values for each ship as travelling on Titanic was more
    expensive.

    Create 3 new variables:
        1. "Fare_scaled" - scale "Fare" columns to have zero mean and standard
       deviation equal one.
        2. "Age_log" - is natural logarithm of attribute "Age" (differences
        between age of children are magnified in comparison to adults).
        3. "Sex" -  Replace string values with numerical ones, where "male"
        will be replaced with 0 and "female" with 1. The resulting values
        should have type `int`.

    Do not modify original DataFrame.
    """
    
    # 1. "Fare_scaled" - scale "Fare" columns to have zero mean and standard
    # deviation equal one.
    df["Fare_scaled"] = (df.Fare-df.Fare.mean())/df.Fare.std()
    
    
    # 2. "Age_log" - is natural logarithm of attribute "Age" (differences
    # between age of children are magnified in comparison to adults).
    df["Age_log"] = numpy.log(df["Age"])

    
    # 3. "Sex" -  Replace string values with numerical ones, where "male"
    # will be replaced with 0 and "female" with 1. The resulting values
    # should have type `int`.
    df["Sex"].replace('female', 1, inplace=True)
    df["Sex"].replace('male', 0, inplace=True)
    
    return df

In [12]:
def determine_survival(df: pd.DataFrame, n_interval: int, age: float,
                       sex: str) -> float:
    """
    Determine the probability of survival of a person specified by age and sex.

    Missing values in column "Age" replace with mean value. In order to
    moderate significance of the estimated probability, divide "Age" to
    specified number of intervals and calculate probability from given
    interval. For example if we have values in "Age" column [2, 13, 18, 25] and
    we want 2 intervals, result should be:

    0    (1.977, 13.5]
    1     (13.5, 25.0]

    With division based on "Sex", the categorization should be:

       "AgeInterval" | "Sex"       |   "Survival Probability"
       (1.977, 13.5] | "male"      |            0.21
       (1.977, 13.5] | "female"    |            0.28
       (13.5, 25.0]  | "male"      |            0.10
       (13.5, 25.0]  | "female"    |            0.15

    Output of determine_survival(df, n_interval=2, age = 5, sex = "male")
    should be 0.21. If there is no passenger for some group, return numpy
    NA value.
    """
    if sex == "male":
        sex = 0
    else: sex = 1
    
    
    data = df.copy()
    # Missing values in column "Age" replace with mean value.
    data.Age.fillna(data.Age.mean(), inplace = True)
    
    age_array = data.Age.unique()
    age_array.sort()
        
    avg = len(age_array) / float(n_interval)
    out = []
    last = 0.0

    while last < len(age_array):
        out.append(age_array[int(last):int(last + avg)])
        last += avg
    
    ###
    display(age_array)
    ###
    display(out)
    
    
    data.drop(columns=["PassengerId", "Pclass", "Name", "SibSp", "Parch", "Fare", "Label", "Fare_scaled", "Age_log"], inplace=True)
    data = data[data.Survived.notna()]
    data.Survived = data.Survived.astype(int)
    
    part = 0
    for i in range(0,n_interval):
        if age in out[part] : break
        part += 1

    count = 0
    ones = 0
    for i in out[part]:
        for j in range(0, len(data.iloc[:,0])):
            if (data.iloc[j,1] == sex) and (i == data.iloc[j,2]):
                if(data.iloc[j,0] == 1):
                    ones += 1
                count += 1
    display(data)
    if count != 0:
        return (ones/count)
    else: return 0
    

In [13]:
# pd.DataFrame data
data = load_dataset("../data/train.csv", "../data/test.csv")
display(data)
# pd.DataFrame df
df = get_missing_values(data)

#pd.DataFrame data_copy
data_copy = substitute_missing_values(data)

#float corr 
corr = get_correlation(data)

# pd.Series series
series = get_survived_per_class(data, "SibSp")

# tuple outliers
outliers = get_outliers(data)

# pd.DataFrame new_features
new_features = create_new_features(data)

# float survival_probability
survaval_probability = determine_survival(data, 2, 5, 'male')
survaval_probability


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Label
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,Train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,Train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,Train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,Train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,Train
...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,8.0500,Test
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,Test
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,Test
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,8.0500,Test


array([ 0.17      ,  0.33      ,  0.42      ,  0.67      ,  0.75      ,
        0.83      ,  0.92      ,  1.        ,  2.        ,  3.        ,
        4.        ,  5.        ,  6.        ,  7.        ,  8.        ,
        9.        , 10.        , 11.        , 11.5       , 12.        ,
       13.        , 14.        , 14.5       , 15.        , 16.        ,
       17.        , 18.        , 18.5       , 19.        , 20.        ,
       20.5       , 21.        , 22.        , 22.5       , 23.        ,
       23.5       , 24.        , 24.5       , 25.        , 26.        ,
       26.5       , 27.        , 28.        , 28.5       , 29.        ,
       29.88113767, 30.        , 30.5       , 31.        , 32.        ,
       32.5       , 33.        , 34.        , 34.5       , 35.        ,
       36.        , 36.5       , 37.        , 38.        , 38.5       ,
       39.        , 40.        , 40.5       , 41.        , 42.        ,
       43.        , 44.        , 45.        , 45.5       , 46.  

[array([ 0.17      ,  0.33      ,  0.42      ,  0.67      ,  0.75      ,
         0.83      ,  0.92      ,  1.        ,  2.        ,  3.        ,
         4.        ,  5.        ,  6.        ,  7.        ,  8.        ,
         9.        , 10.        , 11.        , 11.5       , 12.        ,
        13.        , 14.        , 14.5       , 15.        , 16.        ,
        17.        , 18.        , 18.5       , 19.        , 20.        ,
        20.5       , 21.        , 22.        , 22.5       , 23.        ,
        23.5       , 24.        , 24.5       , 25.        , 26.        ,
        26.5       , 27.        , 28.        , 28.5       , 29.        ,
        29.88113767, 30.        , 30.5       , 31.        ]),
 array([32. , 32.5, 33. , 34. , 34.5, 35. , 36. , 36.5, 37. , 38. , 38.5,
        39. , 40. , 40.5, 41. , 42. , 43. , 44. , 45. , 45.5, 46. , 47. ,
        48. , 49. , 50. , 51. , 52. , 53. , 54. , 55. , 55.5, 56. , 57. ,
        58. , 59. , 60. , 60.5, 61. , 62. , 63. , 64. , 65.

,Survived,Sex,Age
0,0,0,22.000000
1,1,1,38.000000
2,1,1,26.000000
3,1,1,35.000000
4,0,0,35.000000
...,...,...,...
886,0,0,27.000000
887,1,1,19.000000
888,0,1,29.881138
889,1,0,26.000000


0.18393782383419688